In [11]:
%set_env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [12]:
import import_ipynb
import dataset

In [13]:
import tensorflow as tf
import matplotlib.pyplot as plt
import segmentation_models as sm
from datetime import datetime

In [14]:
SEED = 42
tf.random.set_seed(SEED)

In [15]:
data_p_dir = 'data_p/s/'
image_dir = data_p_dir + 'image/'
label_dir = data_p_dir + 'label/'

dataset = dataset.train_dataset(image_dir, label_dir)
dataset = dataset.shuffle(buffer_size=10000, seed=SEED)

split_ratio = 0.8
num_samples = dataset.cardinality().numpy()
num_train = int(split_ratio * num_samples)

# Split the dataset into training and validation sets
train_ds = dataset.take(num_train)
val_ds = dataset.skip(num_train)

In [16]:
model = sm.Unet('efficientnetb2', classes=1, activation='sigmoid')

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
model.compile('adam', sm.losses.DiceLoss(), metrics)

# model.summary()

In [17]:
def make_callbacks():
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

    checkpoint_path = "model_checkpoint.h5"
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        save_best_only=True,
        monitor='val_loss',
        mode='min',
        verbose=1
    )

    return [tensorboard_callback, checkpoint_callback]

In [10]:
BATCH_SIZE = 4
train_ds = train_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=make_callbacks())

Epoch 1/10


2024-02-13 15:47:42.739479: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - ETA: 0s - loss: 0.9790 - iou_score: 0.0135 - f1-score: 0.0266
Epoch 1: val_loss improved from inf to 0.99395, saving model to model_checkpoint.h5
4/4 [==============================] - 24s 3s/step - loss: 0.9790 - iou_score: 0.0135 - f1-score: 0.0266 - val_loss: 0.9940 - val_iou_score: 0.0030 - val_f1-score: 0.0060
Epoch 2/10
4/4 [==============================] - ETA: 0s - loss: 0.9652 - iou_score: 0.0369 - f1-score: 0.0704
Epoch 2: val_loss improved from 0.99395 to 0.97568, saving model to model_checkpoint.h5
4/4 [==============================] - 10s 2s/step - loss: 0.9652 - iou_score: 0.0369 - f1-score: 0.0704 - val_loss: 0.9757 - val_iou_score: 0.0123 - val_f1-score: 0.0243
Epoch 3/10
4/4 [==============================] - ETA: 0s - loss: 0.9656 - iou_score: 0.0605 - f1-score: 0.1133
Epoch 3: val_loss did not improve from 0.97568
4/4 [==============================] - 9s 2s/step - loss: 0.9656 - iou_score: 0.0605 - f1-score: 0.1133 - val_loss